In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import tools
from importlib import reload


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

pd.options.mode.chained_assignment = None  # default='warn'

#### Ronda y fecha de cada ronda
fecha_round_1_matching = '2022_08_11'
fecha_round_2_matching = '2022_08_11'

#### Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)
#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"
path_dropbox = r"C:\Users\oficina825\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_dropbox = r"C:\Users\37615965\Dropbox"

#### Rutas utilizadas (autogeneradas, no tocar):
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_datain = path_dropbox + r"\BID-LGBTQ+\campo\data\para_analisis"
path_dataout = path_dropbox + r"\BID-LGBTQ+\campo\data\mails_matches\matched\correccion_manual_de_matches"
# path_deleted = path_data + r"\mails_matches\deleted"

In [4]:
##################### IMPORTO BASES

#### Levanto la base "original-Full"
print('df_main')
df_main = pd.read_excel(path_datain + fr'\df_final_con_matches_manuales_RA_r1.xlsx')
df_main = tools.limpia_prueba_google_y_quien_sos(df_main, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')

#### Devoluciones de los RA + generación de id_respuesta
reload(tools)
print('\ndf')
identifica_respuestas  = ['thread_id_inmob','fecha','from_inmob']
df = tools.importa_y_genera_ids(
            path_dataout + '\RA_devolucion\Ronda-2', 
            'xlsx', 
            'id_respuesta', 
            identifica_respuestas
     )
df = tools.limpia_prueba_google_y_quien_sos(df, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')
df.sort_values(by='fecha', inplace=True)
df.rename(columns={'Unnamed: 0':'id_merge'}, inplace=True)

#### Devoluciones de los RA pre David + generación de id_respuesta
reload(tools)
print('\ndf_unclean')
identifica_respuestas  = ['thread_id_inmob','fecha','from_inmob']
df_unclean = tools.importa_y_genera_ids(
            path_dataout + '\RA_devolucion\Ronda-2\Pre-David', 
            'xlsx', 
            'id_respuesta', 
            identifica_respuestas
     )
df_unclean = tools.limpia_prueba_google_y_quien_sos(df_unclean, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')
df_unclean.sort_values(by='fecha', inplace=True)

#### Publicaciones enviadas a los RA +  generación de id_publicación
print('\npublicaciones_ra')
identifica_publicacion = ['link','municipio','id','ronda']
publicaciones_ra = tools.importa_y_genera_ids(
            path_dataout + r'\pubs_RA\ronda 2 - not matched', 
            'xlsx', 
            'id_publicacion_hash', 
            identifica_publicacion
     )

df_main
Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 0 de ¿Quien sos?

df
Hay 10 archivos con la extension establecida dentro de la carpeta.
La base tiene 986 observaciones.
Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 11 de ¿Quien sos?

df_unclean
Hay 10 archivos con la extension establecida dentro de la carpeta.
La base tiene 986 observaciones.
Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 0 observaciones de MDS.
Se dropearon 11 de ¿Quien sos?

publicaciones_ra
Hay 10 archivos con la exte

In [19]:
print("ANALISIS DE LAS BASES\n")

tot = len(df)
mds = (df.id_publicacion == 'MDS').sum()
na = df.id_publicacion.isna().sum()
resto = tot - mds - na

print(  "Resultados con la actualización de David:\n"
        "Total  ",len(df),"\n",
        "N/A    ",na,"\n",
        "Valid  ",resto, sep=""   
        )

tot = len(df_unclean)
mds = (df_unclean.id_publicacion == 'MDS').sum()
na = df_unclean.id_publicacion.isna().sum()
resto = tot - mds - na

print(  "\nResultados anteriores:\n"
        "Total  ",len(df_unclean),"\n",
        "N/A    ",na,"\n",
        "Valid  ",resto, sep=""   
        )

ANALISIS DE LAS BASES

Resultados con la actualización de David:
Total  975
N/A    84
Valid  891

Resultados anteriores:
Total  975
N/A    105
Valid  870


In [20]:
# Análisis de la base

q_matched = df.id_publicacion.notnull().sum()
q_not_matched = df.id_publicacion.isnull().sum()
q_total = q_matched+q_not_matched
q_unique = len(df[df.id_publicacion.duplicated(keep='first')==False])

print("\n", q_matched, " publicaciones matcheadas\n", 
      q_not_matched, " publicaciones no matcheadas\n",
      q_total, " total de publicaciones\n",
      round(q_matched / q_total * 100,1), "% porcentaje de publicaciones matcheadas\n", 
      q_unique, " publicaciones unicas", sep='')


891 publicaciones matcheadas
84 publicaciones no matcheadas
975 total de publicaciones
91.4% porcentaje de publicaciones matcheadas
673 publicaciones unicas


In [21]:
df.columns

Index(['id_merge', 'subject_inmob', 'from_inmob', 'to_inmob', 'body_inmob',
       'thread_id_inmob', 'pais', 'fecha', 'RA', 'id_publicacion',
       'Nom Inmobiliaria', 'Comentario', 'Unnamed: 12', 'id_respuesta'],
      dtype='object')

In [22]:
df_main.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'ronda', 'tipo', 'id_publicacion', 'link',
       'id', 'nombre', 'inmobiliaria', 'fecha', 'municipio', 'ciudad', 'pais',
       'descripcion', 'nombre_2', 'precio', 'moneda', 'ambientes',
       'habitaciones', 'banos', 'm2_totales', 'm2_cubiertos', 'cocheras',
       'precio_local', 'p_m2', 'p_m2_above_mdn', 'n_adds', 'n_adds_above_mdn',
       'n_adds_muni', 'n_adds_muni_above_mdn', 'm2_above_mdn', 'common_name',
       'capital', 'random', 'nombre_c', 'precio_c', 'inmobiliaria_c',
       'treatment', 'ses', 'male_name', 'mail', 'male_couple', 'female_couple',
       'script', 'script_string', 'script_string_2', 'randomization_seed',
       'subject_conf', 'from_conf', 'to_conf', 'body_conf', 'date_conf',
       'date_clean_conf', 'date_conf_clean', 'pais_conf', 'match_confiab_conf',
       'match_step_conf', 'id_respuesta', 'subject_inmob', 'from_inmob',
       'to_inmob', 'body_inmob', 'date_inmob', 'cc_inmob', 'thread_id_inmob',
       'mimeTy

In [23]:
# Acomodo columnas
df = df.drop(columns=['Unnamed: 12'])
df = df.rename(columns={'Nom Inmobiliaria':'RA_inmob_name',
                   'Comentario':'RA_match_comment',
                   'fecha':'date_inmob_clean'
     })

# Construyo columna "tipo"
df['tipo'] = np.where(df['id_publicacion'].isnull(), 'solo_respuesta', 'match_ambas')

# Construyo columna 'match_manual'
df['match_manual'] = 1

## Merge RA con base main

In [24]:
### Reconstruyo id_publicacion de la base RA
# Para ello, tengo que mergear la base publicaciones (que les enviamos a los ra y de donde obtienen la columna id_publicacion) con 
# la base de categorizaciones de RA. De la base publicaciones_ra solo necesito las columnas ['id_publicacion','id_publicacion_hash']
df = publicaciones_ra[['id_publicacion','id_publicacion_hash']].merge(df, on='id_publicacion', indicator=True)
print(df._merge.value_counts())
df = df.drop(columns=['id_publicacion','_merge'])
df = df.rename(columns={'id_publicacion_hash':'id_publicacion'})
df = tools.normaliza_columnas_post_merge(df, suffixes=('_x','_y'))
                                         
publicaciones_ra = publicaciones_ra.drop(columns=['id_publicacion'])
publicaciones_ra = publicaciones_ra.rename(columns={'id_publicacion_hash':'id_publicacion'})

both          891
left_only       0
right_only      0
Name: _merge, dtype: int64


In [25]:
vars_publicacion = ['ronda','id_publicacion','link','id','nombre','inmobiliaria','fecha','municipio','ciudad','descripcion','nombre_2','precio','moneda','ambientes','habitaciones','banos','m2_totales','m2_cubiertos','cocheras','precio_local','p_m2','p_m2_above_mdn','n_adds','n_adds_above_mdn','n_adds_muni','n_adds_muni_above_mdn','m2_above_mdn','common_name','capital','random','nombre_c','precio_c','inmobiliaria_c','treatment','ses','male_name','mail','male_couple','female_couple','script','script_string','script_string_2','randomization_seed']
vars_conf_envio  = ['subject_conf','from_conf','to_conf','body_conf','date_conf','date_clean_conf','date_conf_clean','pais_conf','match_confiab_conf','match_step_conf']
vars_rtas_inmob  = ['id_respuesta','subject_inmob','from_inmob','to_inmob','body_inmob','date_inmob','cc_inmob','thread_id_inmob','mimeType_inmob','pais_inmob','date_clean_inmob','date_inmob_clean','match_confiab_inmob','match_step_inmob','match_string_inmob','match2_string_inmob']

### FIXME: ¿Qué pasó con estas 4 observaciones no categorizadas? ¿Dónde las perdimos?

reload(tools)
df_final, test_rtas, test_pubs, new_matched = tools.incorpora_matches_manuales(df_main, 
                                                                        rtas_ra=df, 
                                                                        pubs_ra=publicaciones_ra, 
                                                                        ronda=2, 
                                                                        vars_pubs=vars_publicacion+vars_conf_envio, 
                                                                        vars_rtas=vars_rtas_inmob
                                            ) 
df_final.to_excel(path_datain + fr'\df_final_con_matches_manuales_RA_r1_y_r2.xlsx')
print(path_datain + fr'\df_final_con_matches_manuales_RA_r1_y_r2.xlsx', "creado")

Hay respuestas que fueron categorizadas por RA y no están en la base main. Son las siguientes:


,id_respuesta,id_publicacion_y,id_merge_y,subject_inmob_y,from_inmob_y,to_inmob_y,body_inmob_y,thread_id_inmob_y,pais_y,date_inmob_clean_y,RA_y,RA_inmob_name_y,RA_match_comment_y,tipo_y,match_manual_y,_merge
2248,1d21365dc005b5f5697c345d642a97d88428af94b63e50...,47d129e6f6c1d0534a0382adefae3613ed49dc0cd482fc...,406.0,RE: Te consultaron por la propiedad Avenida Ca...,inmuebles@difonzoprop.com.ar,rodrigue.zmanuel.w@gmail.com,"[<p>Hola buen dia, el departamento esta dispon...",180943edd709277d,arg,2022-05-05 09:43:11,analiariveroborges@gmail.com,NaN,NaN,match_ambas,1.0,right_only
2249,e586e6183d812d21eba253db14a482d86633d8ee66b8ab...,b72c7b00495b34d7a6a7719e5786c36ec1247648ffec6d...,418.0,Alquiler,Silvana Romussi <Ricardo Portal@gmail.com>,rodrigue.zmanuel.w@gmail.com,[<p>Hola Manuel está vigente el alquiler es 2 ...,180946f5bae5422c,arg,2022-05-05 10:35:55,analiariveroborges@gmail.com,NaN,parece que fuera el id 500 nombre martin alons...,match_ambas,1.0,right_only
2250,4d9cb0f52c22292ae95aa01b4f5c3266c37bc79d4143df...,16d10a9369a200e55a955947c5cffe9dde9c8efd16a5c0...,881.0,Respuesta a tu consulta - zoegrupoinmobiliari,zoegrupoinmobiliari - Contacto <b059bc9e9373c2...,j.danielflores.n@gmail.com,[<p>@import url(http://url2214.inmomail.com/ls...,180a70bb5c871bec,per,2022-05-09 01:20:02,rodriffernandez@gmail.com,zoe negocios inmobiliarios,NaN,match_ambas,1.0,right_only
2251,4d9cb0f52c22292ae95aa01b4f5c3266c37bc79d4143df...,16d10a9369a200e55a955947c5cffe9dde9c8efd16a5c0...,880.0,Respuesta a tu consulta - zoegrupoinmobiliari,zoegrupoinmobiliari - Contacto <b059bc9e9373c2...,j.danielflores.n@gmail.com,[<p>@import url(http://url2214.inmomail.com/ls...,180a70bb5c871bec,per,2022-05-09 01:20:02,rodriffernandez@gmail.com,zoe negocios inmobiliarios,NaN,match_ambas,1.0,right_only


883
571
C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\para_analisis\df_final_con_matches_manuales_RA_r1_y_r2.xlsx creado


In [26]:
len(df_final) 
# 9810

8678

### Revisiones y otros export

In [27]:
data = test_rtas[test_rtas['_merge']=='right_only'][['from_inmob_y','to_inmob_y']]
for mail_from, mail_to in data.to_records(index=False):
    filtered = df_main[(df_main['from_inmob']==mail_from) & (df_main['to_inmob']==mail_to)]
    if len(filtered)>0:
        print("La observación", df_main[df_main['from_inmob']==mail_from].index, "de la base df_main contiene el mail", mail_from)

    else:
        print("La observación de", mail_from, "no se encuentra en df_main")

La observación de inmuebles@difonzoprop.com.ar no se encuentra en df_main
La observación de Silvana Romussi <Ricardo Portal@gmail.com> no se encuentra en df_main
La observación Int64Index([8030, 8031, 8032], dtype='int64') de la base df_main contiene el mail zoegrupoinmobiliari - Contacto <b059bc9e9373c269af67c863ec68ea6d_0.12088.0.1003863.rsp8041231@inmomail.com>
La observación Int64Index([8030, 8031, 8032], dtype='int64') de la base df_main contiene el mail zoegrupoinmobiliari - Contacto <b059bc9e9373c269af67c863ec68ea6d_0.12088.0.1003863.rsp8041231@inmomail.com>


In [28]:
pd.crosstab(df_final['RA'],df_final['tipo'])

tipo,match_ambas
RA,
anaguadalupedepasqua@hotmail.com,215
analiariveroborges@gmail.com,187
canuelobelen@gmail.com,217
castano.fernando.martin@gmail.com,215
euge.basile.eb@gmail.com,85
euge.basile.eb@gmail.com,69
manuelfmb12@gmail.com,193
marianomenendez0@gmail.com,93
marianomenendez0@gmail.com,76


In [29]:
pd.crosstab(df_final.tipo,df_final.ronda)

ronda,1,2
tipo,,
match_ambas,2165,2205
solo_publicacion,2072,2057
solo_respuesta,95,84


In [30]:
test = df_final.dropna(subset='from_inmob')
test[test['from_inmob'].str.contains('gabriel@interwin.com.ar')]

,Unnamed: 0.1,Unnamed: 0,ronda,tipo,id_publicacion,link,id,nombre,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,subject_inmob,from_inmob,to_inmob,body_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,Unnamed: 4,id_pub2,RA_inmob_name,RA_match_comment
8446,2831.0,NaN,1,match_ambas,48c84f0d12386f48b36e54e0bc262d83d217d79ff91a51...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,departamento en alquiler en oro al 3000,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",departamento en alquiler en oro al 3000,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333300,1.0,17.0,1.0,16.0,1.0,1.0,2.0,1.0,0.5768,oro 3000,250000.0,interwin marketing,hetero,neutral,Manuel,jf.manuel.rodriguez@gmail.com,Juan,Laura,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Laura.,23.0,Hiciste una consulta por la propiedad Oro Al 3000,Properati <contacts@mailing.properati.com.ar>,jf.manuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 20:04:26 +0000 (UTC)",2022-04-19,2022-04-19 17:04:26,arg,NaN,NaN,ce2dc7885cac0f79d97f7883e42a994b13c4552348c118...,oro,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",jf.manuel.rodriguez@gmail.com,[<p>sldos_x000D_\n_x000D_\ngabriel_x000D_\n_x0...,NaN,NaN,1804374b86d3e037,NaN,NaN,NaT,2022-04-19 17:12:43,NaN,NaN,NaN,NaN,1.0,369.0,silvataniaelizabeth@gmail.com,NaN,999.0,interwin marketing inmobiliario,NaN
5037,NaN,NaN,2,match_ambas,1d7378f53ebbb64aa6117d094445b86a8ed06a0d1ef001...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,departamento en alquiler en oro al 3000,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",NaN,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333252,1.0,16.0,1.0,NaN,NaN,1.0,2.0,1.0,NaN,oro 3000,250000.0,interwin marketing,trans,high,Pablo,f.h.pablogomez@gmail.com,Luis,Florencia,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Florencia (ella es trans),2.0,Hiciste una consulta por la propiedad Oro Al 3000,Properati <contacts@mailing.properati.com.ar>,f.h.pablogomez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 04 May 2022 17:29:06 +0000 (UTC)",2022-05-04,2022-05-04 14:29:06,NaN,NaN,NaN,30bf331b087a74bac24e02fa96859c920c4a98ae26f43a...,ALQUILER ORO TODO RECICLADO A NUEVO,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",f.h.pablogomez@gmail.com,"[<p>ALQUILER ORO TODO RECICLADO A NUEVO , EL ...",NaN,NaN,180904e64528f81f,NaN,NaN,NaT,2022-05-04 15:21:36,NaN,NaN,NaN,NaN,1.0,679.0,canuelobelen@gmail.com,NaN,NaN,interwin marketing inmobiliario,NaN
5038,NaN,NaN,2,match_ambas,1d7378f53ebbb64aa6117d094445b86a8ed06a0d1ef001...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,departamento en alquiler en oro al 3000,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",NaN,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.333252,1.0,16.0,1.0,NaN,NaN,1.0,2.0,1.0,NaN,oro 3000,250000.0,interwin marketing,trans,high,Pablo,f.h.pablogomez@gmail.com,Luis,Florencia,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Florencia (ella es trans),2.0,Hiciste una consul

In [31]:
# Base para Juli. Solo respuestas + ronda 1.
df_juli = df_final[(df_final['tipo']=='solo_respuesta') & (df_final['ronda']==2)]
df_juli = df_juli.loc[:,'subject_inmob':]
df_juli.to_excel(path_dataout + fr'\mensajes_vacios\no_matcheados_ronda2_v2.xlsx')

In [32]:
import pandas as pd
df_test = pd.read_excel(path_dataout + fr'\bases_operativas\df_final_con_matches_manuales_RA.xlsx')